In [ ]:
!pip -q install langchain langgraph langchain_core langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.2 MB/s eta 0:00:00


In [ ]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

os.environ['LANGSMITH_API_KEY'] =  userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = "new-project"
os.environ["LANGSMITH_TRACING_V2"] = "true"

In [ ]:
@tool
def search(query: str):
    """Call to surf the web."""
    if "gn" in query.lower() or "gurgaon" in query.lower():
        return "It's 25 degrees celcius and foggy."
    return "It's 90 degrees and sunny."

tools = [search]
tool_node = ToolNode(tools)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

def call_model(state: MessagesState):
    messages = state['messages']
    # Invoking `model` will automatically infer the correct tracing context
    response = model.invoke(messages)
    return {"messages": [response]}


In [ ]:
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge("__start__", "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", 'agent')

app = workflow.compile()

In [ ]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in gurgaon")]},
    config={"configurable": {"thread_id": 42}}
)

In [ ]:
final_state["messages"][-1].content

'The current weather in Gurgaon is 60 degrees Fahrenheit and foggy.'